In [ ]:
%matplotlib inline

import io
from collections import OrderedDict
import datetime
import pytz

import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt

# plt.style.use('ggplot')
plt.style.use('fivethirtyeight')

from IPython.display import Markdown, display, HTML

def printhtml(string):
    display(HTML(string))

def printmd(string):
    display(Markdown(string))

tables = ['ZPILLOWUSER', 'ZSLEEPNOTE', 'Z_2SLEEPSESSION', 'ZSLEEPSESSION', 'ZSLEEPSTAGEDATAPOINT', 'ZSNOOZELAB', 'ZSOUNDDATAPOINT', 'Z_PRIMARYKEY', 'Z_METADATA', 'Z_MODELCACHE', 'Y_UBMETA', 'Y_UBRANGE', 'Y_UBKVS']
# tables = ['ZSLEEPSESSION', 'ZSLEEPSTAGEDATAPOINT']

def getDictFromString(string):
    tokens = string.split()
    row = OrderedDict()
    
    while tokens:
        value_parts = [tokens.pop()]
        value_part_or_sep = tokens.pop()  # ' -> '
        while value_part_or_sep != '->':
            value_parts.append(value_part_or_sep)
            value_part_or_sep = tokens.pop()
        value_parts.reverse()
        
        key = tokens.pop()
        row[key] = ' '.join(value_parts)
        
    return row

filename = 'PillowData.txt'
reading_table = None
dataframes = {}
rows = []

with open(filename) as file:
    for line in file:
        line = line.strip()
        
        if line in tables:
            if reading_table:
                dataframes[reading_table] = pd.DataFrame(rows)
            reading_table = line
            rows = []
            continue
        elif line == '':
            dataframes[reading_table] = pd.DataFrame(rows)
            break
        
        rows.append(getDictFromString(line))

# Reverse the order of the columns in each dataframe.
for k, v in dataframes.items():
    dataframes[k] = v.iloc[:, ::-1]

# Convert each column to numeric datatype if possible.
for df in dataframes.values():
    for col in df.columns:
        try:
            df[col] = pd.to_numeric(df[col])
        except:
            pass

df_sessions = dataframes['ZSLEEPSESSION']
df_stages = dataframes['ZSLEEPSTAGEDATAPOINT']
df_audio = dataframes['ZSOUNDDATAPOINT']
df_session_notes = dataframes['Z_2SLEEPSESSION']
df_notes = dataframes['ZSLEEPNOTE']

def makeDateTime(timestamp, year=2018):
    return datetime.datetime.fromtimestamp(timestamp, pytz.timezone('US/Eastern')).replace(year=year)

df_stages.sort_values('ZTIMESTAMP', inplace=True)
df_stages['ZTIMESTAMP'] = df_stages['ZTIMESTAMP'].apply(makeDateTime)
df_sessions['ZSTARTTIME'] = df_sessions['ZSTARTTIME'].apply(makeDateTime)
df_sessions['ZENDTIME'] = df_sessions['ZENDTIME'].apply(makeDateTime)
df_audio.sort_values('ZTIMESTAMP', inplace=True)
df_audio['ZTIMESTAMP'] = df_audio['ZTIMESTAMP'].apply(makeDateTime)

audio_color = 'purple'
colors=['dodgerblue', 'mediumaquamarine', 'deeppink', 'darkorange']

def plotSession(session):
    df = df_stages[df_stages['ZSLEEPSESSION']==session]
    dfa = df_audio[df_audio['ZSLEEPSESSION']==session]

    x = df['ZTIMESTAMP'].values
    y = df['ZSLEEPSTAGE'].values

    plt.close('all')
    fig = plt.figure(figsize=(10, 4))
    ax = fig.add_subplot(111)

    for row in dfa.iterrows():
        ax.bar(row[1]['ZTIMESTAMP'], 4, 5*np.timedelta64(row[1]['ZDURATION'], 's')/np.timedelta64(1, 'D'), 0, align='edge', color=audio_color)

    left = x[0]
    width = datetime.timedelta(0)
    bottom = y[0]
    height = 1
    for i, val in enumerate(y):
        if val != bottom:
            ax.bar(left, height, width, bottom, align='edge', color=colors[int(bottom)])
            left = x[i]
            bottom = val
            width = datetime.timedelta(0)
        width = (x[i] - left) / np.timedelta64(1, 'D')

    ax.bar(left, height, width, bottom, align='edge', color=colors[int(bottom)])

    ax.set_yticks([0, 1, 2, 3, 4])
    ax.set_yticklabels(['', 'Deep', 'Light', 'REM', 'Awake'])

    plt.tick_params(axis='y', which='minor', bottom=False, top=False, labelbottom=False)

    ax.xaxis.set_major_formatter(matplotlib.dates.DateFormatter('%I:%M %p', tz=pytz.timezone('US/Eastern')))

    fig.tight_layout()
    output = io.StringIO()
    plt.savefig(output, format="svg")
    plt.close()
    return output.getvalue()[214:-1]

def plotStackedPercentages(*args):
    plt.close('all')
    fig = plt.figure(figsize=(10, 1))
    ax = fig.add_subplot(111)
    
    height = 1
    left = 0
    
    for i, c in enumerate(reversed(colors)):
        width = args[i]
        ax.bar(left, height, width, 0, align='edge', color=c)
        left += width
        
    plt.axis('off')
    ax.set_facecolor(None)
    
    fig.tight_layout()
    output = io.StringIO()
    plt.savefig(output, format="svg", transparent=True)
    plt.close()
    return output.getvalue()[214:-1]

moods = ['Bad', 'Not so good', 'OK', 'Good', 'Great']
mood_emojis = ['😧', '🙁', '😐', '🙂', '😀']

def roundTimedelta(t):
    if hasattr(t, 'total_seconds'):
        return datetime.timedelta(seconds=round(t.total_seconds()))
    return datetime.timedelta(seconds=round(t))

# List to store session data
session_data = []

def showSession(row):
    session = row['Z_PK']
    when = row['ZSTARTTIME'].strftime('%A, %B %d, %Y at %I:%M %p')
    total_time = row['ZENDTIME'] - row['ZSTARTTIME']
    duration = str(roundTimedelta(total_time))
    time_to_sleep = roundTimedelta(row['ZTIMEAWAKE'])
    asleep = roundTimedelta(total_time.total_seconds() - row['ZTIMEAWAKE'])
    quality = row['ZSLEEPQUALITY']
    mood = row['ZWAKEUPMOOD'] - 1
    mood_string = '{emoji} {description}'.format(emoji=mood_emojis[mood], description=moods[mood])
    
    rem = round(roundTimedelta(row['ZTIMEINREMSLEEP'])/total_time*100)
    light = round(roundTimedelta(row['ZTIMEINLIGHTSLEEP'])/total_time*100)
    deep = round(roundTimedelta(row['ZTIMEINDEEPSLEEP'])/total_time*100)
    awake = 100 - (rem + light + deep)
    
    notes = []
    for note in df_session_notes[df_session_notes['Z_3SLEEPSESSION']==session].iterrows():
        note_pk = note[1]['Z_2SLEEPNOTE']
        note_text = df_notes[df_notes['Z_PK']==note_pk].iloc[0].at['ZCONTENTTEXT']
        notes.append(note_text)
    
    printmd('# {when}'.format(when=when))
    
    stages_graph = plotSession(session)
    printhtml(stages_graph)
    
    table = """|  | | | |
|------------:|:-----|----:|:----|
| Time in bed | {duration} | Sleep quality |{quality:.0%} |
| Time asleep | {asleep} | Mood | {mood} |
| Time to sleep | {time_to_sleep} | Notes | {notes}&nbsp; |"""
    printmd(table.format(when=when,
                         duration=duration, 
                         asleep=asleep, 
                         time_to_sleep=time_to_sleep,
                         quality=quality, 
                         mood=mood_string,
                         notes='\n'.join(notes)
                        ))
    
    printhtml(plotStackedPercentages(awake, rem, light, deep))

    colored_dots = ['<font style="color:{color}">\u2B24</font>'.format(color=c) for c in colors]
    table_percents = """| | | | |
|-----------:|-----:|----:|----:|
| Awake | {awake}%{dota} | REM | {rem}%{dotr} |
| Light Sleep | {light}%{dotl} | Deep Sleep | {deep}%{dotd} |"""
    printmd(table_percents.format(awake=awake,
                                  rem=rem,
                                  light=light,
                                  deep=deep,
                                  dotd=colored_dots[0],
                                  dotl=colored_dots[1],
                                  dotr=colored_dots[2],
                                  dota=colored_dots[3]
                                 ))
    
    # Collect session data
    session_data.append({
        'Session': session,
        'When': when,
        'Duration': duration,
        'Time Asleep': str(asleep),
        'Time to Sleep': str(time_to_sleep),
        'Sleep Quality': f'{quality:.0%}',
        'Mood': mood_string,
        'REM (%)': rem,
        'Light (%)': light,
        'Deep (%)': deep,
        'Awake (%)': awake,
        'Notes': '\n'.join(notes)
    })

# Process each session and display
for _, session_row in df_sessions.iterrows():
    showSession(session_row)

# Save session data to CSV
session_df = pd.DataFrame(session_data)
csv_filename = 'session_data.csv'
session_df.to_csv(csv_filename, index=False)
print(f"Session data saved to {csv_filename}")
